In [9]:
import os
from dotenv import load_dotenv
from io import BytesIO
from elevenlabs.client import ElevenLabs

load_dotenv(verbose=True)

True

In [24]:
# ファイルロード
file_path = r"../data/raw/4人の男女の会話.mp3"
with open(file_path, "rb") as f:
    audio_data = BytesIO(f.read())

In [25]:
client = ElevenLabs(
  api_key=os.environ['ELEVENLABS_API_KEY'],
)

In [26]:
transcription = client.speech_to_text.convert(
    file=audio_data,
    model_id="scribe_v1",  # モデルの指定。現時点では "scribe_v1" のみサポート
    tag_audio_events=True, # 笑いや拍手などの音声イベントのタグ付けを有効化
    language_code="jpn",   # 音声の言語。指定がない場合は自動判別
    diarize=True,          # 話者ダイアライぜーションのアノテーションを有効化
)

In [27]:
def format_time(t):
    """
    秒数 t を "mm:ss.mmm" 形式の文字列に変換する
    """
    minutes = int(t // 60)
    seconds = t - minutes * 60
    return f"{minutes:02d}:{seconds:06.3f}"

# セグメント毎にまとめるためのリスト
segments = []
current_segment = None
gap_threshold = 0.5  # 次の単語との間隔がこの秒数以上ならセグメントを分割

for w in transcription.words:
    # 各単語の情報（辞書形式を想定）
    w = dict(w)
    speaker = w.get("speaker_id", "unknown")
    start = w.get("start", 0)
    end = w.get("end", 0)
    text_piece = w.get("text", "")
    
    # current_segment が未定義の場合は新規作成
    if current_segment is None:
        current_segment = {"speaker": speaker, "start": start, "end": end, "text": text_piece}
    else:
        # 話者が変わるか、または前の単語とのギャップが大きい場合はセグメントを終了する
        if speaker != current_segment["speaker"] or (start - current_segment["end"]) > gap_threshold:
            segments.append(current_segment)
            current_segment = {"speaker": speaker, "start": start, "end": end, "text": text_piece}
        else:
            # 同一セグメントに追加：end 時刻を更新し、テキストを連結
            current_segment["end"] = end
            current_segment["text"] += text_piece

# ループ後、残ったセグメントを追加
if current_segment is not None:
    segments.append(current_segment)

# 各セグメントを指定のフォーマットで出力する
for seg in segments:
    start_str = format_time(seg["start"])
    end_str = format_time(seg["end"])
    # speaker_id が "speaker_0" の場合は SPEAKER_00 のように整形
    spk = seg["speaker"]
    if spk.startswith("speaker_"):
        number = spk.split("_")[1]
        spk_label = f"SPEAKER_{int(number):02d}"
    else:
        spk_label = spk.upper()
    print(f"[{start_str} - {end_str}] {spk_label}: {seg['text']}")


[00:00.280 - 00:04.619] SPEAKER_00: こんにちは！シェアハウスコルサってここですか？ 
[00:04.619 - 00:07.399] SPEAKER_01: もしかして、四人目の住人の人？ 
[00:07.399 - 00:15.059] SPEAKER_00: はい、吉田有理です。よろしくお願いします。思ったよりいいとこですね、ここ。 
[00:15.059 - 00:19.119] SPEAKER_01: そうでしょ？僕チョウです。 
[00:19.119 - 00:24.239] SPEAKER_02: やっと来た。ユリ今何時だと思ってる？ 
[00:24.239 - 00:28.579] SPEAKER_00: ごめんごめん、スティーブつい二度寝しちゃって。 
[00:28.579 - 00:30.219] SPEAKER_02: せめて電話ぐらい。 
[00:30.219 - 00:33.799] SPEAKER_00: はい。本当にごめんなさい。 
[00:33.799 - 00:39.059] SPEAKER_01: 二度寝気持ちいいんですよね。僕もよくやっちゃう。 
[00:39.059 - 00:46.000] SPEAKER_03: 共有スペースはもう片付いたから、そこにある吉田さんの荷物を自分の部屋に運んでくれる？ 
[00:46.000 - 00:48.259] SPEAKER_00: あ、ユリでいいです。 
[00:48.259 - 00:52.020] SPEAKER_03: じゃあユリさん、お願いね。私はアナ。 
[00:52.020 - 00:56.939] SPEAKER_00: アナさん、よろしくお願いします。


In [28]:
print(transcription.text)

こんにちは！シェアハウスコルサってここですか？ もしかして、四人目の住人の人？ はい、吉田有理です。よろしくお願いします。思ったよりいいとこですね、ここ。 そうでしょ？僕チョウです。 やっと来た。ユリ今何時だと思ってる？ ごめんごめん、スティーブつい二度寝しちゃって。 せめて電話ぐらい。 はい。本当にごめんなさい。 二度寝気持ちいいんですよね。僕もよくやっちゃう。 共有スペースはもう片付いたから、そこにある吉田さんの荷物を自分の部屋に運んでくれる？ あ、ユリでいいです。 じゃあユリさん、お願いね。私はアナ。 アナさん、よろしくお願いします。


In [29]:
for w in transcription.words:
    print(w.model_dump())

{'text': 'こ', 'start': 0.28, 'end': 0.359, 'type': 'word', 'speaker_id': 'speaker_0', 'characters': None}
{'text': 'ん', 'start': 0.359, 'end': 0.459, 'type': 'word', 'speaker_id': 'speaker_0', 'characters': None}
{'text': 'に', 'start': 0.459, 'end': 0.579, 'type': 'word', 'speaker_id': 'speaker_0', 'characters': None}
{'text': 'ち', 'start': 0.579, 'end': 0.719, 'type': 'word', 'speaker_id': 'speaker_0', 'characters': None}
{'text': 'は', 'start': 0.719, 'end': 0.879, 'type': 'word', 'speaker_id': 'speaker_0', 'characters': None}
{'text': '！', 'start': 0.879, 'end': 1.599, 'type': 'word', 'speaker_id': 'speaker_0', 'characters': None}
{'text': 'シ', 'start': 1.599, 'end': 1.659, 'type': 'word', 'speaker_id': 'speaker_0', 'characters': None}
{'text': 'ェ', 'start': 1.659, 'end': 1.759, 'type': 'word', 'speaker_id': 'speaker_0', 'characters': None}
{'text': 'ア', 'start': 1.759, 'end': 1.899, 'type': 'word', 'speaker_id': 'speaker_0', 'characters': None}
{'text': 'ハ', 'start': 1.899, 'end': 2